## GMF
![image.png](imgs/GMF.png)

GMF 是基于 MF 的，并且在 MF 的基础上提供了扩展。

神经网络的参数我们可以看做是对用户、物品隐藏特征的加权，非线性的激活函数（如sigmoid函数）可以捕捉传统 MF 所不能学习的非线性特征。

user 和 item 都通过 one-hot 编码得到稀疏向量，然后通过一个 embedding 层映射为 user vector 和 item vector。

这样就获得了 user 和 item 的隐向量，就可以使用哈达玛积(Element-wise Produce)来交互 user 和 item 向量了。

![image.png](imgs/GMF公式.png)

![image.png](imgs/哈达玛积.png)

### 伪代码
```python
# input
user = Input(shape=(1,))
item = Input(shape=(1,))

# embedding
user_embedding = Embedding(input_dim, output_dim, 1)(user)
item_embedding= Embedding(input_dim, output_dim, 1)(item)

# GMF
vector = multiply([user_embedding, item_embedding])

# output
prediction = Dense(1, 'sigmoid')(vector)

model = Model(inputs=[user, item], outputs=prediction)
```

## MLP(Multi Layer Perceptron)
![image.png](imgs/MLP.png)
MLP 虽然不能像 GMF 那样还原传统的 MF，但是 MLP 的多层神经网络设计，能够让其学习到比 GMF 更复杂的用户、物品交互特征。

user 和 item 都通过 one-hot 编码得到稀疏向量，然后通过一个 embedding 层映射为 user vector 和 item vector。然后将 user 和 item vector 输入到 MLP 中进行训练，MLP 大家都知道，重点是如何处理 user 和 item vector，作者是直接将两个向量串联起来（多模态中的做法，这在多模态中是一种常见的做法，还可以用其它方式，比如简单串联后加正则，或者找个建模能力更强的神经网络模型）。

这样串联起来后，输入到 MLP 中，可以学习出 user 和 item 之间的非线性交互函数。具体MLP过程是这么做的：

![image.png](imgs/MLP公式.png)

### 伪代码
```python
# input
user = Input(shape=(1,))
item = Input(shape=(1,))

# embedding
user_embedding = Embedding(input_dim, output_dim, 1)(user)
item_embedding= Embedding(input_dim, output_dim, 1)(item)

# MLP
vector = concatenate([user_embedding, item_embedding])
layers = [emb1,emb2, emb3]
for i in range(len(layers)):
    vector = Dense(layers.index(i))

# output
prediction = Dense(1, 'sigmoid')(vector)

model = Model(inputs=[user, item], outputs=prediction)
```

## NeuMF
NeuMF是NCF的第三个实现，它结合了GMF和MLP，吸取了二者的优势，可以比GMF和MLP更好的表示用户、物品交互行为特征

![image.png](imgs/NCF.png)

NeuMF的左侧是 GMF，右侧是 MLP，然后再将二者的输出做 Concatenation，最后再通过一层神经网络来得到最终的评分预测。

GMF 因为是对 MF 的扩展，因此可以比较好的捕捉用户、物品之间的线性关系

MLP 由于多层神经网络的贡献，可以学习更多的用户、物品之间非线性关系

NeuMF 通过对二者的结合，既能很好的学习线性关系，也能很好的学习非线性关系，从而做出更准确的评分预测。

![image.png](imgs/NCF公式.png)
### 伪代码
```python
# input
user = Input(shape=(1,))
item = Input(shape=(1,))

# embedding
mf_user_embedding = Embedding(input_dim, output_dim1, 1)(user)
mf_item_embedding= Embedding(input_dim, output_dim1, 1)(item)

mlp_user_embedding = Embedding(input_dim, output_dim2, 1)(user)
mlp_item_embedding= Embedding(input_dim, output_dim2, 1)(item)

# GMF
mf_vector = multiply([mf_user_embedding, mf_item_embedding])

# MLP
mlp_vector = concatenate([mlp_user_embedding, mlp_item_embedding])
layers = [emb1,emb2, emb3]
for i in range(len(layers)):
    mlp_vector = Dense(layers.index(i))(mlp_vector)

# output
predict_vector = concatenate([mf_vector, mlp_vector])
prediction = Dense(1, activation='sigmoid')(predict_vector)

model = Model(inputs=[user, item], outputs=prediction)
```
## 显性反馈和隐性反馈
NCF专注于专注于隐性反馈，可以通过用户对item的行为，比如购买产品和点击项目等间接反映用户偏好的行为。无论你是评过分，收藏过，看过有关内容，都可以作为隐性反馈。与显性反馈（评分和评级）不同，隐性反馈可以更容易收集。

对于有评分这样的显性反馈，有评分就用，没有评分的数据也要处理，一般就是记分值为0。而对于隐性反馈，为了处理没有观察到的数据，可以将所有无交互的条目视为负反馈，也可以从中抽样作为负反馈实例，NCF 中也记为 0。

令 user_num 和 item_num 分别表示用户和项目的数量，通过从用户的隐性反馈学习 user-item 交互矩阵：

![image.png](imgs/NCF的标签.png)

这里 y 为 1 表示用户 u 和项目 item 存在交互记录，然而这并不意味着 u 真的喜欢 i。同样的，值 0 也不是表明 u 不喜欢 i，也有可能是这个用户根本不知道有这个项目。

这对隐性反馈的学习提出了挑战，因为它提供了关于用户偏好的噪声信号。